#Extra llevado a cabo por Joel Brenes Vargas.

## Interoperabilidad

El siguiente codigo esta generado por chatGPT con el siguiente prompt:

Prompt (joel.brenes.vargas@est.una.ac.cr.com):

"Genera un ejemplo de interoperabilidad entre python y c++ (usando pybind11) para mejorar el rendimiento de una operacion pesada. El programa debe incluir una version en python con operaciones costosas y una operacion equivalente en c++. Al final compara los tiempos de corrida."

In [6]:
!pip install pybind11

In [1]:
%%writefile setup.py
from setuptools import setup
from pybind11.setup_helpers import Pybind11Extension, build_ext

ext_modules = [
    Pybind11Extension(
        "sum_arrays",
        ["sum_arrays.cpp"],
    ),
]

setup(
    name="sum_arrays",
    ext_modules=ext_modules,
    cmdclass={"build_ext": build_ext},
)


Overwriting setup.py


In [2]:
%%writefile sum_arrays.cpp
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
#include <cmath>

namespace py = pybind11;

py::array_t<double> sum_arrays_cpp(py::array_t<double> a, py::array_t<double> b) {
    auto buf1 = a.request(), buf2 = b.request();
    if (buf1.size != buf2.size)
        throw std::runtime_error("Arrays must be the same size");

    auto result = py::array_t<double>(buf1.size);
    auto buf3 = result.request();

    double* ptr1 = static_cast<double*>(buf1.ptr);
    double* ptr2 = static_cast<double*>(buf2.ptr);
    double* ptr3 = static_cast<double*>(buf3.ptr);

    for (ssize_t i = 0; i < buf1.size; i++) {
        double acc = 0;
        for (int j = 0; j < 50; j++) {
            acc += std::sqrt(ptr1[i] + ptr2[i] + j);
        }
        ptr3[i] = acc;
    }

    result.resize({buf1.shape[0]});
    return result;
}

PYBIND11_MODULE(sum_arrays, m) {
    m.def("sum_arrays_cpp", &sum_arrays_cpp, "Sum two large arrays in C++");
}


Overwriting sum_arrays.cpp


In [3]:
!rm sum_arrays*.so
!python setup.py build_ext --inplace


running build_ext
x86_64-linux-gnu-g++ -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/include/python3.11 -c flagcheck.cpp -o flagcheck.o -std=c++17
building 'sum_arrays' extension
x86_64-linux-gnu-g++ -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.11/dist-packages/pybind11/include -I/usr/include/python3.11 -c sum_arrays.cpp -o build/temp.linux-x86_64-cpython-311/sum_arrays.o -std=c++17 -fvisibility=hidden -g0
x86_64-linux-gnu-g++ -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -shared -Wl,-O1 -Wl,-Bsymbolic-functions build/temp.linux-x86_64-cpython-311/sum_arrays.o -L/usr/lib/x86_64-linux-gnu -o build/lib.linux-x86_64-cpython-311/sum_arrays.cpython-311-x86_64-linux-gnu.so
copying build/lib.linux-x86_64-cpython-311/sum_arrays.

In [4]:
import random
import time
import math
from sum_arrays import sum_arrays_cpp

# Generar listas grandes de floats
size = 1_000_000
a = [random.random() for _ in range(size)]
b = [random.random() for _ in range(size)]

# Función en Python puro
def suma_pura(a, b):
    result = [0] * len(a)
    for i in range(len(a)):
        acc = 0
        for j in range(50):
            acc += math.sqrt(a[i] + b[i] + j)
        result[i] = acc
    return result

# Función para comparar listas
def all_close(list1, list2, tol=1e-6):
    if len(list1) != len(list2):
        return False
    for x, y in zip(list1, list2):
        if abs(x - y) > tol:
            return False
    return True

# Medir tiempo Python puro
start_py = time.time()
res_py = suma_pura(a, b)
end_py = time.time()
print(f"Python puro: {end_py - start_py:.4f} s")

# Medir tiempo C++
import numpy as np
a_np = np.array(a, dtype=np.float64)
b_np = np.array(b, dtype=np.float64)

start_cpp = time.time()
res_cpp = sum_arrays_cpp(a_np, b_np)
end_cpp = time.time()
print(f"C++ vía pybind11: {end_cpp - start_cpp:.4f} s")

# Validar resultados
print("¿Resultados iguales?", all_close(res_py, res_cpp.tolist()))

for i in range(10):
    print(f"{i}: Python={res_py[i]:.8f}, C++={res_cpp[i]:.8f}, Δ={abs(res_py[i] - res_cpp[i]):.10f}")




Python puro: 6.8339 s
C++ vía pybind11: 0.1186 s
¿Resultados iguales? True
0: Python=243.38012313, C++=243.38012313, Δ=0.0000000000
1: Python=237.00890427, C++=237.00890427, Δ=0.0000000000
2: Python=240.34765917, C++=240.34765917, Δ=0.0000000000
3: Python=241.90554424, C++=241.90554424, Δ=0.0000000000
4: Python=239.04414849, C++=239.04414849, Δ=0.0000000000
5: Python=235.96714735, C++=235.96714735, Δ=0.0000000000
6: Python=242.98258245, C++=242.98258245, Δ=0.0000000000
7: Python=239.89223307, C++=239.89223307, Δ=0.0000000000
8: Python=241.57815342, C++=241.57815342, Δ=0.0000000000
9: Python=242.81380692, C++=242.81380692, Δ=0.0000000000


## Profiling

In [8]:
"""
Prompt (joel.brenes.vargas@est.una.ac.cr.com):
Genera un código en Python altamente demostrativo del uso de profiling, con un cuello de botella evidente y medible mediante cProfile. El programa debe permitir analizar y cuantificar el rendimiento de diferentes partes del código al generar una gran cantidad de objetos y realizar operaciones costosas.
"""

import numpy as np
import cProfile
import pstats

def generate_matrices(n):
    """Genera dos matrices n×n con valores aleatorios en [0,1)."""
    return np.random.rand(n, n), np.random.rand(n, n)

def matmul_naive(A, B):
    """
    Multiplicación de matrices ingenua usando listas de Python.
    A y B deben ser listas de listas (no arrays de NumPy).
    """
    n = len(A)
    # Inicializar resultado con ceros
    C = [[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            total = 0.0
            for k in range(n):
                total += A[i][k] * B[k][j]
            C[i][j] = total
    return C

def matmul_numpy(A, B):
    """Multiplicación de matrices usando NumPy (dot product)."""
    return A.dot(B)

def profile_function(func, *args):
    """
    Ejecuta func(*args) bajo cProfile y muestra las 10 funciones
    más costosas ordenadas por tottime.
    """
    profiler = cProfile.Profile()
    profiler.enable()
    result = func(*args)
    profiler.disable()

    stats = pstats.Stats(profiler).sort_stats('tottime')
    stats.print_stats(10)
    return result

def main():
    n = 300  # Tamaño de las matrices (ajusta según tu entorno)
    print(f"Generando matrices {n}×{n}...")
    A, B = generate_matrices(n)

    print("\n--- Profiling: implementación ingenua ---")
    # matmul_naive necesita listas de Python
    profile_function(matmul_naive, A.tolist(), B.tolist())

    print("\n--- Profiling: implementación NumPy ---")
    profile_function(matmul_numpy, A, B)

if __name__ == "__main__":
    main()


Generando matrices 300×300...

--- Profiling: implementación ingenua ---
         4 function calls in 7.959 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    7.958    7.958    7.959    7.959 /tmp/ipython-input-8-3909244992.py:14(matmul_naive)
        1    0.001    0.001    0.001    0.001 /tmp/ipython-input-8-3909244992.py:21(<listcomp>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}



--- Profiling: implementación NumPy ---
         3 function calls in 0.015 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.015    0.015    0.015    0.015 {method 'dot' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.015    0.015 /tmp/ipython-input-8-3909244992.py:30(matmul_numpy)
        1    0.000    0.000    0.000    